In [ ]:
# # from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
# # import pandas as pd
# # import numpy as np

# # # load your 20 columns pandas dataset as X_train and your 5 columns pandas dataset as y_train
# # X_train = pd.read_csv("InputTrain.csv").iloc[:,2:]
# # # reshape X_train to be a list of time series data
# # X_train = [X_train.iloc[:, i].values.reshape(-1, 1) for i in range(X_train.shape[1])]
# # X_train = np.concatenate(X_train, axis=1)

# # # convert y_train to a numpy array
# # y_train = pd.read_csv('StepOne_LabelTrain.csv').iloc[:,2]
# # y_train = y_train.values
# # # create the KNN classifier with k=1
# # knn = KNeighborsTimeSeriesClassifier(n_neighbors=1)

# # # fit the classifier to the training data
# # knn.fit(X_train, y_train)

# # # make predictions on new data
# # # reshape your new data (X_test) in the same way as X_train
# # X_test = pd.read_csv("InputTest.csv").iloc[:,2:]
# # X_test = [X_test.iloc[:, i].values.reshape(-1, 1) for i in range(X_test.shape[1])]
# # X_test = np.concatenate(X_test, axis=1)
# # y_pred = knn.predict(X_test)



# from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier

# dfX = pd.read_csv("InputTrain.csv").iloc[:,2:]
# # melt the dataframe to create a new dataframe with one column
# new_df = pd.melt(dfX)
# X = new_df["value"].values
# Y = pd.read_csv("StepOne_LabelTrain.csv").iloc[:,2].to_numpy()
# knn = KNeighborsTimeSeriesClassifier(n_neighbors=1)
# X_train, X_test, y_train, y_test = train_test_split(X, Y)

# knn.fit(X_train, y_train)
# y_pred = knn.predict(X_test)
# accuracy = knn.score(X_test, y_test)


ValueError: Found input variables with inconsistent numbers of samples: [22509360, 10421]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.tree import DecisionTreeClassifier
from sktime.classification.interval_based import TimeSeriesForestClassifier
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
from sktime.datasets import load_arrow_head
# transformers in scikit-learn maybe the solution to this 
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from sklearn.dummy import DummyClassifier

In [ ]:

import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.tree import DecisionTreeClassifier
from sktime.classification.interval_based import TimeSeriesForestClassifier
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
from sktime.datasets import load_arrow_head
# transformers in scikit-learn maybe the solution to this 
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from sklearn.dummy import DummyClassifier


import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.tree import DecisionTreeClassifier
from sktime.classification.interval_based import TimeSeriesForestClassifier
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
from sktime.datasets import load_arrow_head
# transformers in scikit-learn maybe the solution to this 
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from sklearn.dummy import DummyClassifier

In [31]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
scaler = MinMaxScaler()
import pandas as pd

In [156]:
import numpy as np

import math
import re
import pywt

X = pd.read_csv("InputTrain.csv").iloc[:,2:]
y = pd.read_csv("StepOne_LabelTrain.csv").iloc[:,2:]
dfXTest = pd.read_csv("InputTest.csv").iloc[:,2:]
#l'ajout manuel des features prend un peu de temps c'est pas opti, ~50sec pour la cellule entière

# Ajout manuel de 3 features (qui utilisent des Series et pas le dataframe entier)
autocor = []
idmax = []
entropy = []
for i in range(0,10421) :
  autocor.append(X.iloc[i,0:2161].autocorr(216))
  idmax.append((int)(re.search(r'\d+',X.iloc[i,0:2161].idxmax())[0]))
  entropy.append(sum([ p * math.log(p) / math.log(2.0) for p in X.iloc[i,0:2161]]))
# X['autocor'] = autocor
X['idmax'] = idmax
X['entropy'] = entropy
autocor = []
idmax = []
entropy = []
print(dfXTest.shape)
for i in range(0,2488) :
  autocor.append(dfXTest.iloc[i,0:2161].autocorr(216))
  idmax.append((int)(re.search(r'\d+',dfXTest.iloc[i,0:2161].idxmax())[0]))
  entropy.append(sum([ p * math.log(p) / math.log(2.0) for p in dfXTest.iloc[i,0:2161]]))
# dfXTest['autocor'] = autocor
dfXTest['idmax'] = idmax
dfXTest['entropy'] = entropy


# Ajout automatique de 6 features (qui utilisent le dataframe entier)
X['mean'] = X.iloc[:, 2:2161].mean(axis=1)

X['median'] = X.iloc[:, 2:2161].median(axis=1)

X['std'] = X.iloc[:, 2:2161].std(axis=1)

X['var'] = X.iloc[:, 2:2161].var(axis=1)

X['skew'] = X.iloc[:, 2:2161].skew(axis=1)

X['sem'] = X.iloc[:, 2:2161].sem(axis=1)


dfXTest['mean'] = dfXTest.iloc[:, 2:2161].mean(axis=1)

dfXTest['median'] = dfXTest.iloc[:, 2:2161].median(axis=1)

dfXTest['std'] = dfXTest.iloc[:, 2:2161].std(axis=1)

dfXTest['var'] = dfXTest.iloc[:, 2:2161].var(axis=1)

dfXTest['skew'] = dfXTest.iloc[:, 2:2161].skew(axis=1)

dfXTest['sem'] = dfXTest.iloc[:, 2:2161].sem(axis=1)


def wavelet_variance(row):
    coeffs = pywt.wavedec(row, 'db4', level=5) # perform 5-level decomposition using db4 wavelet
    variances = [c.var() for c in coeffs] # calculate the variances of each detail coefficient
    
    return sum(variances) # return the sum of variances

X['WaveletVar'] = X.apply(wavelet_variance, axis=1)
dfXTest['WaveletVar'] = dfXTest.apply(wavelet_variance, axis=1)



# scaled_x = scaler.fit_transform(X)
# scaled_T = scaler.fit_transform(dfXTest)



(2488, 2160)


In [157]:
import xgboost as xgb
from sklearn.model_selection import KFold

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

dtrain = xgb.DMatrix(X_train,y_train)
dtest = xgb.DMatrix(X_test,y_test)

model = xgb.XGBModel(n_estimators=10)
params = {
    'max_depth': 20,
    'eta': 0.3,
    'objective': 'binary:logistic',
    'eval_metric': 'rmse',
    'booster':'gbtree',
    'subsample': 0.85,
    'learning_rate':0.2,

}

# Train model
evalist = [(dtrain, 'train'), (dtest, 'eval')]
model = xgb.train(params, dtrain,evals=evalist)

dFin = xgb.DMatrix(dfXTest)

preds = model.predict(dFin) #Predictions





[0]	train-rmse:0.42286	eval-rmse:0.43974
[1]	train-rmse:0.36280	eval-rmse:0.39682
[2]	train-rmse:0.31510	eval-rmse:0.36568
[3]	train-rmse:0.27602	eval-rmse:0.34432
[4]	train-rmse:0.24436	eval-rmse:0.32921
[5]	train-rmse:0.21739	eval-rmse:0.31854
[6]	train-rmse:0.19466	eval-rmse:0.31107
[7]	train-rmse:0.17602	eval-rmse:0.30569
[8]	train-rmse:0.16061	eval-rmse:0.30180
[9]	train-rmse:0.14689	eval-rmse:0.29906


In [151]:
dfF = pd.DataFrame(preds, columns=["Washing Machine","Dishwasher","Tumble Dryer","Microwave","Kettle"])
dfF = dfF.rename_axis('Index')
dfF.to_csv("oneT.csv")

In [ ]:

# totF =0
# for k in range (1,5) :
#     dfX['std'] = dfX.iloc[:, 2:].std(axis=1)
#     dfX['var'] = dfX.iloc[:, 2:].var(axis=1)
#     total = 0

#     for i in range(2,7):
#         dfY = pd.read_csv("StepOne_LabelTrain.csv").iloc[:,i].to_numpy()
#         X_train, X_test, y_train, y_test = train_test_split(dfX, dfY)
#         classifier = DummyClassifier(strategy="prior")
#         classifier.fit(X_train, y_train)
#         total += classifier.score(X_test, y_test)

#     totF += total /5
# print(totF/4)


0.8317920184190329


In [93]:
# df = pd.DataFrame(preds,columns=["Washing Machine","Dishwasher","Tumble Dryer","Microwave","Kettle"])
# y_test
# # acc_rfc = (df == y_test).sum().astype(float) / len(preds)*100

# # df.to_csv("XGB1.csv")
# # y_test
# df

,Washing Machine,Dishwasher,Tumble Dryer,Microwave,Kettle
0,0.196328,0.155483,0.072301,0.170945,0.278544
1,0.158500,0.265667,0.021269,0.106424,0.365963
2,0.052177,0.043402,0.006387,0.056172,0.227191
3,0.161630,0.136407,0.040873,0.139150,0.290110
4,0.228074,0.159977,0.176064,0.239632,0.400129
...,...,...,...,...,...
2080,0.266665,0.185597,0.015363,0.181303,0.251275
2081,0.192667,0.160788,0.074388,0.180723,0.280159
2082,0.325879,0.309132,0.199683,0.301394,0.374529
2083,0.250574,0.226559,0.093387,0.164121,0.278832


In [94]:
# y_test

,Washing Machine,Dishwasher,Tumble Dryer,Microwave,Kettle
8233,0,0,0,0,0
5860,0,0,0,0,0
3463,0,0,0,0,0
3911,0,0,0,0,1
6246,0,0,0,0,1
...,...,...,...,...,...
7035,0,0,0,0,0
4207,0,0,0,0,0
6663,0,0,0,0,0
10409,1,0,0,0,0


In [ ]:

# df.to_csv("XGB1.csv")

In [ ]:
import xgboost as xgb

# create a DMATRIX from the training data

# create an XGBoost model
xgb_model = xgb.XGBModel()

# train the model on the DMATRIX
xgb_model.fit(dtrain)

# use the model to make predictions on the test data


In [72]:
# y_pred
preds
df = pd.DataFrame(preds)